# Persiapan

## Install Firmware Micropython

* Untuk berinteraksi dengan esp, menggunakan esptool. Gunakan pip, bukan pip3
```
$ pip install esptool
```

* Bisa dilihat apakah sudah terinstall atau belum
```
$ which esptool.py
/usr/local/bin/esptool.py
```
* atau langsung coba
```
$ esptool.py
usage: esptool [-h] [--chip {auto,esp8266,esp32}] [--port PORT] [--baud BAUD]
  [--before {default_reset,no_reset}]
  [--after {hard_reset,soft_reset,no_reset}] [--no-stub]
  [--trace]
  {load_ram,dump_mem,read_mem,write_mem,write_flash,run,image_info,make_image,elf2image,read_mac,chip_id,flash_id,read_flash_status,write_flash_status,read_flash,verify_flash,erase_flash,erase_region,version}
  ...
esptool: error: too few arguments
```

* Sambungkan board, dan lihat pada port berapa board tersebut akan dimunculkan
```
$ ls /dev/ | grep USB
ttyUSB0
```

* Lalu kita hapus firmware yang masih ada digantikan dengan firmware micropython
```
$ esptool.py --port /dev/ttyUSB0 erase_flash
```

* Download firmware micropython, cari yang esp8266 dengan ekstensi .bin di http://micropython.org/download. Atau bisa juga menggunakan wget
```
$ wget http://micropython.org/resources/firmware/esp8266-20171101-v1.9.3.bin
```

* Selanjutnya, install firmware pada board

```
$ esptool.py --port /dev/ttyUSB0 --baud 460800 write_flash --flash_size=detect 0 esp8266-20171101-v1.9.3.bin
```

* Untuk mengujinya, bisa menggunakan sejenis tool serial monitor, disini menggunakan CuteCom

![](img/cutecom.png)

**Perlu diingat, sambungan serial hanya bisa dibuka oleh satu aplikasi dalam satu waktu.**

## Jupyter untuk Micropython

* Pastikan sudah memiliki jupyter pada sistem
``` 
$ pip3 install jupyter
```

* Clone kernel micropython di https://github.com/adafruit/jupyter_micropython_kernel
```
$ git clone https://github.com/adafruit/jupyter_micropython_kernel.git
```

* Sekarang buat pengaturan micropython pada jupyter
```
$ cd ~/.local/share/jupyter/kernel
$ mkdir micropython
$ cd micropython
$ nano kernel.json
```
* Akan muncul editor nano pada terminal, copy dan paste kode dibawah ini lalu simpan (**ctrl+o** lalu **ctrl+x**)
```json
{
 "argv": ["python3",
          "-m", "jupyter_micropython_kernel",
          "/dev/ttyUSB0", "115200",
          "-f", "{connection_file}"],
 "display_name": "MicroPython - /dev/ttyUSB0",
 "language": "micropython"
}
```

* Selanjutnya, ketikan `jupyter notebook` dan kernel mycropython akan ditambahkan
![](img/micropython.png)



## AMPY

* Install dalam sistem
```
$ sudo pip3 install adafruit-ampy
```

* Test apakah sudah berhasil di install

```
$ ampy
ampy
Usage: ampy [OPTIONS] COMMAND [ARGS]...

  ampy - Adafruit MicroPython Tool

  Ampy is a tool to control MicroPython boards over a serial
  connection.  Using ampy you can manipulate files on the board's
  internal filesystem and even run scripts.

Options:
  -p, --port PORT    Name of serial port for connected board.  Can
                     optionally specify with AMPY_PORT environemnt
                     variable.  [required]
  -b, --baud BAUD    Baud rate for the serial connection (default
                     115200).  Can optionally specify with AMPY_BAUD
                     environment variable.
  -d, --delay DELAY  Delay in seconds before entering RAW MODE (default
                     0). Can optionally specify with AMPY_DELAY
                     environment variable.
  --version          Show the version and exit.
  --help             Show this message and exit.

Commands:
  get    Retrieve a file from the board.
  ls     List contents of a directory on the board.
  mkdir  Create a directory on the board.
  put    Put a file or folder and its contents on the...
  reset  Perform soft reset/reboot of the board.
  rm     Remove a file from the board.
  rmdir  Forcefully remove a folder and all its...
  run    Run a script and print its output.

```

* Penggunaan
```
$ ampy --port /dev/ttyUSB0 ls
boot.py
```

* Agar lebih nyaman, masukan port dalam PATH (masukan kedalam .bashrc juga)
```
$ export AMPY_PORT=/dev/ttyUSB0
$ ampy ls
```

## Menguji Micropython

In [1]:
import machine
import time

# Input dan Output pada Micropython

Micropython | Board
--|--
0|D3
2|D4 (also Led1 but inverse)*
4|D2
5|D1
9|SD2
10|SD3
12|D6
13|D7
14|D5
15|D8
16|D0 (also Led2 but inverse)*

## Output Digital

State output:
* 0 = Low = 0 Volt
* 1 = High = 5 Volt / 3.3 Volt

In [7]:
led1 = machine.Pin(2, machine.Pin.OUT)
led2 = machine.Pin(16, machine.Pin.OUT)

In [16]:
led1.value(0)

In [17]:
led2.value(0)

In [18]:
led1.value(1)

In [19]:
led2.value(1)

# Tabel XOR untuk Toggle

X|Y|z
--|--|--
0|0|0
0|1|1
1|0|1
1|1|0

```python
state = 0
state = state xor 1    # 0 xor 1 hasilnya 1
state = state xor 1    # 1 xor 1 hasilnya 0
```

In [20]:
led_state = 0
led_state = led_state ^ 1
print(led_state)

1


In [21]:
led_state = led_state ^ 1
print(led_state)

0


In [23]:
led_state = 0

for i in range(0,10): 
    led1.value(led_state)
    led_state = led_state ^ 1
    led2.value(led_state)
    
    time.sleep(1)

## Output Analog (PWM)

In [24]:
led1 = machine.Pin(2, machine.Pin.OUT)
dim = machine.PWM(led1)

* Duty Cycle: 0 - 1023
* LED konfigurasi active low 
    * angka duty cycle semakin besar maka semakin redup, karena tegangannya mengecil

In [25]:
dim.freq(500)
dim.duty(512)

In [26]:
for i in range(0,5):
    for j in range(0,1024):
        time.sleep_us(700) 
        dim.duty(1024-j)
        
    time.sleep_ms(700) 
    
    for j in range(0,1024):
        time.sleep_us(700) 
        dim.duty(j)

        
    time.sleep(2)        

## Input Digital

In [27]:
button = machine.Pin(0, machine.Pin.IN, machine.Pin.PULL_UP)

In [29]:
print(button.value())

0


## Input Analog (ADC)

In [31]:
adc = machine.ADC(0)

In [33]:
on = 0
off = 1

for i in range(0,1023):
    value = adc.read()
    print(adc.read())
    if value < 40:
        led2.value(on)
    else :
        led2.value(off)
    time.sleep_ms(5)

171
123
187
127
185
149
168
173
126
186
125
186
147
172
169
137
185
123
186
137
177
162
150
181
122
187
133
180
159
158
178
121
175
161
153
180
121
187
133
182
153
162
175
124
187
127
184
150
168
171
130
186
124
186
147
170
171
129
186
125
186
148
171
169
135
185
124
186
141
176
163
149
181
121
187
134
180
158
159
176
124
186
148
171
169
136
186
124
186
144
174
168
142
182
122
187
136
178
160
154
178
120
187
130
182
153
162
176
121
188
128
182
153
163
173
124
186
126
184
148
171
170
133
185
123
186
142
175
163
149
181
121
187
137
175
163
148
181
118
186
135
178
158
156
177
120
186
133
180
153
161
174
121
186
127
183
149
164
173
124
186
125
185
145
171
166
136
182
122
185
136
173
158
149
174
116
182
128
177
154
156
176
117
170
158
132
175
116
181
135
172
161
147
178
118
184
136
174
160
149
177
117
183
132
171
169
112
170
116
154
126
121
132
87
121
82
99
77
72
68
48
53
37
39
29
28
24
20
20
18
19
17
17
16
15
16
14
17
15
14
16
15
16
14
17
16
17
18
17
22
21
29
30
36
48
39
66
53
84
80
90
114

## Kombinasi PWM dan ADC

In [34]:
import machine
import time

In [35]:
led1 = machine.Pin(2, machine.Pin.OUT)
dim = machine.PWM(led1)
dim.freq(500)

In [36]:
adc = machine.ADC(0)

In [37]:
brightness = 0

In [39]:
for i in range(0,1023):
    value = adc.read()
    print(value)
    if value < 40:
        brightness += 50
        if brightness > 1023:
            brightness = 0
        time.sleep_ms(100) 
        dim.duty(1023-brightness)
    time.sleep_ms(5)
            

176
144
182
122
188
127
187
139
181
156
169
171
144
182
122
188
128
187
140
181
157
168
183
130
185
123
188
132
184
150
174
165
154
177
125
187
124
188
134
184
150
174
164
156
177
127
186
124
188
134
175
150
173
165
152
180
121
187
126
187
139
181
153
169
170
148
181
122
187
126
187
137
182
153
170
169
149
180
126
189
127
185
141
176
168
149
180
122
187
125
186
138
181
157
168
171
145
182
121
188
126
187
139
178
160
162
172
136
182
121
183
133
183
149
172
168
150
178
122
186
125
186
128
181
151
170
166
149
178
124
187
130
183
145
171
150
145
159
118
178
123
177
133
181
149
170
162
150
175
123
184
123
185
134
180
150
169
165
136
165
115
172
115
163
120
147
122
123
123
96
116
77
101
72
90
65
72
56
52
40
32
16
16
14
14
15
14
13
21
13
17
12
18
12
14
16
10
10
12
14
12
13
10
89
116
105
115
115
89
98
57
67
48
76
67
103
98
120
133
118
151
106
171
115
172
128
164
150
150
166
120
176
114
180
124
176
137
170
156
152
170
125
178
117
182
126
181
142
175
161
151
175
123
175
128
181
147
172
162
151
1

## LED RGB

In [40]:
import machine
import time

In [41]:
red = machine.Pin(13, machine.Pin.OUT)
green = machine.Pin(14, machine.Pin.OUT)
blue = machine.Pin(12, machine.Pin.OUT)

on = 0
off = 1

In [42]:
red = machine.PWM(red)
red.freq(500)
green = machine.PWM(green)
green.freq(500)
blue = machine.PWM(blue)
blue.freq(500)

In [43]:
red.duty(1023)
green.duty(1023)
blue.duty(1023)

In [45]:
for i in range(0,1023):
    green.duty(i)
    blue.duty(1023-i)
    time.sleep_ms(5)
time.sleep_ms(500)    

for i in range(0,1023):
    blue.duty(i)
    red.duty(1023-i)
    time.sleep_ms(5)
time.sleep_ms(500)    
    
for i in range(0,1023):
    red.duty(i)
    green.duty(1023-i)
    time.sleep_ms(5)
time.sleep_ms(500)        

# Wifi pada Micropython

In [46]:
import network

In [47]:
sta_if = network.WLAN(network.STA_IF)

In [48]:
print(sta_if.active())

True


In [49]:
print(sta_if.ifconfig())

('0.0.0.0', '0.0.0.0', '0.0.0.0', '208.67.222.222')


In [50]:
sta_if.active(True)

In [51]:
sta_if.connect('Yang Lain Aja', 'namadepanmu')

In [52]:
print(sta_if.active())

True


In [54]:
print(sta_if.ifconfig())

('192.168.43.253', '255.255.255.0', '192.168.43.1', '192.168.43.1')


## Request ke internet

In [55]:
import urequests

### Think Speak
* Update a Channel Feed

In [57]:
response = urequests.get('https://api.thingspeak.com/update?api_key=6B9U5HUJFHTJ7FWI&field1=500')
print(response.text)

12


In [58]:
response = urequests.get('https://api.thingspeak.com/channels/455622/feeds.json?api_key=GJJ1RKBMYGWZOVBP&results=2')
print(response.text)

{"channel":{"id":455622,"name":"LampController","latitude":"0.0","longitude":"0.0","field1":"state","created_at":"2018-03-22T00:07:12Z","updated_at":"2018-03-23T12:53:47Z","last_entry_id":12},"feeds":[{"created_at":"2018-03-23T12:53:12Z","entry_id":11,"field1":"100"},{"created_at":"2018-03-23T12:53:47Z","entry_id":12,"field1":"500"}]}


In [59]:
print(response.json()['channel'])

{'field1': 'state', 'id': 455622, 'name': 'LampController', 'updated_at': '2018-03-23T12:53:47Z', 'last_entry_id': 12, 'latitude': '0.0', 'longitude': '0.0', 'created_at': '2018-03-22T00:07:12Z'}


In [60]:
print(response.json()['feeds'][-1])

{'created_at': '2018-03-23T12:53:47Z', 'field1': '500', 'entry_id': 12}


### Our Own Back End

* Registrasi nama

In [63]:
username = 'daus'
# host = 'http://192.168.0.7:5000'
host = 'http://tegar.pythonanywhere.com'
path = '/lamp/api/'

In [64]:
data = {
    "name":username
}
response = urequests.post(host + path + 'reg/', headers={"Content-Type": "application/json"}, json = data)
print(response.text)

{
  "name": {
    "data": [], 
    "name": "daus"
  }
}



* Registrasi Perangkat

In [65]:
import esp
print(esp.flash_id())

1458208


In [66]:
data = {
    "id":esp.flash_id(),
    "location":"bathroom",
    "state":0
}

url = host + path + username+ '/'

response = urequests.post(url, 
                          headers={"Content-Type": "application/json"},
                          json = data)
print(response.text)

{
  "lamp_data": {
    "id": 1458208, 
    "location": "bathroom", 
    "state": 0
  }
}



* Melihat semua lampu yang tersedia

In [67]:
response = urequests.get(url)
print(response.text)

{
  "lamp_data": [
    {
      "id": 1458208, 
      "location": "bathroom", 
      "state": 0
    }
  ]
}



* Melihat lampu dengan ID tertentu

In [70]:
response = urequests.get(url+str(esp.flash_id())+'/')
print(response.text)

{
  "lamp_data": {
    "id": 1458208, 
    "location": "bathroom", 
    "state": 1
  }
}



In [69]:
data = {
    "state":1
}

response = urequests.put(url, 
                          headers={"Content-Type": "application/json"},
                          json = data)
print(response.text)

{
  "lamp": [
    {
      "id": 1458208, 
      "location": "bathroom", 
      "state": 1
    }
  ]
}



In [71]:
data = {
    "id":esp.flash_id(),
    "location":"Living room",
    "state":1
}

url = url + str(esp.flash_id()) + '/'
response = urequests.put(url, 
                          headers={"Content-Type": "application/json"},
                          json = data)
print(response.text)

{
  "lamp": {
    "id": 1458208, 
    "location": "Living room", 
    "state": 1
  }
}

